# Understanding TensorRT - Convert Graph

## Convert the Model
1. checkpoint -> Frozen TensorFlow graph  
2. Frozen TensorFlow graph -> TFLite  
3a. TFLite -> EdgeTPU (Google Coral)  
3b. TFLite -> TensorRT (NVIDIA GPU)  

### Did you train locally or via SageMaker Docker?
- Locally - the training checkpoint is in code/model
- SageMaker - the training checkpoint was pushed to S3 then pulled back down and extracted to /trained_model

The conversion script expects it to be in /trained_model


## CUDA / TensorFlow Versions
### TF Lite
No big deal and there are probably no concerns here

### TensorRT 7.0
TensorRT 7.0 says the model must be trained w/ TensorFlow 1.14 (which requires CUDA 10.0).  
- did you train w/ 1.14  
- did you convert to tflite w/ 1.14  

just beware

### Edge TPU (Google Coral)
Hmm, this all worked with TF 1.13 and 1.14, haven't tested 1.15

In [1]:
import os
import sys

In [2]:
import tensorflow as tf

print (tf.__version__)

1.14.1


/home/jayson/anaconda3/envs/tf114/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jayson/anaconda3/envs/tf114/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jayson/anaconda3/envs/tf114/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jayson/anaconda3/envs/

## GLOBALS

In [3]:
# GLOBALS so work with the bash scripts
PROJECT_DIR = os.getcwd()
TASKS_DIR = os.path.join(PROJECT_DIR, 'tasks')
CODE_DIR = os.path.join(PROJECT_DIR, 'code')
TFLITE_DIR = os.path.join(PROJECT_DIR, 'tflite_model')
TENSORFLOW_DIR = os.path.join(PROJECT_DIR, 'tensorflow_model')
MODEL_RESEARCH_DIR = os.path.join(PROJECT_DIR, 'code/models/research')
SLIM_DIR = os.path.join(MODEL_RESEARCH_DIR, 'slim')
TRAINED_DIR = os.path.join(PROJECT_DIR, 'trained_model')

NUM_TRAINING_STEPS = 6000
# Local (non-SageMaker) -or- SageMaker
# PIPELINE_CONFIG = 'sagemaker_mobilenet_v1_ssd_retrain.config'
PIPELINE_CONFIG = 'local_mobilenet_v1_ssd_retrain.config'

# PICK 1 - A. Local or  B. Hosted or C. Original 

## A - Locally Trained
### get the checkpoint files to the correct location

In [ ]:
# is your checkpoint (NUM_TRAINING_STEPS) there?
! ls {CODE_DIR}/model/*{NUM_TRAINING_STEPS}*

In [ ]:
# copy to the /trained_model directory
# this will make it compatible with the convert script
! cp {CODE_DIR}/model/*{NUM_TRAINING_STEPS}* {TRAINED_DIR}
! ls {TRAINED_DIR}/*{NUM_TRAINING_STEPS}* -l

## check the configuration

In [ ]:
# PIPELINE_CONFIG = 'sagemaker_mobilenet_v1_ssd_retrain.config'
PIPELINE_CONFIG = 'local_mobilenet_v1_ssd_retrain.config'

# verify
! ls {CODE_DIR}/{PIPELINE_CONFIG}

## B - Trained in SageMaker

In [ ]:
! ls {TRAINED_DIR}/*{NUM_TRAINING_STEPS}* -l

In [ ]:
PIPELINE_CONFIG = 'sagemaker_mobilenet_v1_ssd_retrain.config'
# PIPELINE_CONFIG = 'local_mobilenet_v1_ssd_retrain.config'

# verify
! ls {CODE_DIR}/{PIPELINE_CONFIG}

## C - Convert Original MobileNet to tflite_graph.pb
The original MobileNet SSD model - that this project trained on top of - is stored in ssd-dag/code/ckpt.  

If you want to use this model (instead of detect Chick-fil-A products) you first need to convert it convert to a TensorFlow Frozen Graph or a TFLite graph.  Assuming you put the MobileNet in code/ckpt (per the instructions in TrainModel_Step1_Local) - this will set up the environment variables for you to do that very easily.

In [4]:
! cp {CODE_DIR}/ckpt/pipeline.config {CODE_DIR}/local_mobilenet_v1_ssd_coco.config
PIPELINE_CONFIG='local_mobilenet_v1_ssd_coco.config'

NUM_TRAINING_STEPS='0'
! cp {CODE_DIR}/ckpt/model.ckpt.data-00000-of-00001 {TRAINED_DIR}/model.ckpt-0.data-00000-of-00001
! cp {CODE_DIR}/ckpt/model.ckpt.index {TRAINED_DIR}/model.ckpt-0.index
! cp {CODE_DIR}/ckpt/model.ckpt.meta {TRAINED_DIR}/model.ckpt-0.meta

# Convert the Checkpoint to Frozen Graph & TF Lite Graph

In [5]:
# convert checkpoint is a task script - located in the tasks/ directory
os.chdir(TASKS_DIR)  
! ./convert_checkpoint_to_edgetpu_tflite.sh --checkpoint_num {NUM_TRAINING_STEPS} --pipeline_config {PIPELINE_CONFIG}


TASKS_DIR=/home/jayson/projects/ssd-dag/tasks
***
/home/jayson/projects/ssd-dag/tflite_model
:/home/jayson/projects/ssd-dag/code/models/research/slim:/home/jayson/projects/ssd-dag/code/models/research
+ ckpt_number=0
+ [[ 4 -gt 0 ]]
+ case "$1" in
+ ckpt_number=0
+ shift 2
+ [[ 2 -gt 0 ]]
+ case "$1" in
+ pipeline_config=local_mobilenet_v1_ssd_coco.config
+ shift 2
+ [[ 0 -gt 0 ]]
+ rm /home/jayson/projects/ssd-dag/tensorflow_model -rf
+ rm /home/jayson/projects/ssd-dag/tflite_model -rf
+ echo '-- check for model checkpoint (the raw graph):,' , 0
-- check for model checkpoint (the raw graph):, , 0
+ ls /home/jayson/projects/ssd-dag/trained_model/model.ckpt-0.data-00000-of-00001 /home/jayson/projects/ssd-dag/trained_model/model.ckpt-0.index /home/jayson/projects/ssd-dag/trained_model/model.ckpt-0.meta
/home/jayson/projects/ssd-dag/trained_model/model.ckpt-0.data-00000-of-00001
/home/jayson/projects/ssd-dag/trained_model/model.ckpt-0.index
/home/jayson/projects/ssd-dag/trained_model/mode

W0210 18:25:06.415826 139689548273472 ag_logging.py:145] Entity <bound method Conv.call of <tensorflow.python.layers.convolutional.Conv2D object at 0x7f0bb4fe30f0>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Conv.call of <tensorflow.python.layers.convolutional.Conv2D object at 0x7f0bb4fe30f0>>: AssertionError: Bad argument number for Name: 3, expecting 4
W0210 18:25:06.434131 139689548273472 ag_logging.py:145] Entity <bound method BatchNormalization.call of <tensorflow.python.layers.normalization.BatchNormalization object at 0x7f0bb4a27c18>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Batch

W0210 18:25:06.622115 139689548273472 ag_logging.py:145] Entity <bound method BatchNormalization.call of <tensorflow.python.layers.normalization.BatchNormalization object at 0x7f0bb4d5c128>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method BatchNormalization.call of <tensorflow.python.layers.normalization.BatchNormalization object at 0x7f0bb4d5c128>>: AssertionError: Bad argument number for Name: 3, expecting 4
W0210 18:25:06.662733 139689548273472 ag_logging.py:145] Entity <bound method Conv.call of <tensorflow.python.layers.convolutional.Conv2D object at 0x7f0bb4d5c9e8>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: conv

W0210 18:25:06.906570 139689548273472 ag_logging.py:145] Entity <bound method Conv.call of <tensorflow.python.layers.convolutional.Conv2D object at 0x7f0bb4b68c18>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Conv.call of <tensorflow.python.layers.convolutional.Conv2D object at 0x7f0bb4b68c18>>: AssertionError: Bad argument number for Name: 3, expecting 4
W0210 18:25:06.923691 139689548273472 ag_logging.py:145] Entity <bound method BatchNormalization.call of <tensorflow.python.layers.normalization.BatchNormalization object at 0x7f0bb4aeb080>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Batch

W0210 18:25:07.143311 139689548273472 ag_logging.py:145] Entity <bound method BatchNormalization.call of <tensorflow.python.layers.normalization.BatchNormalization object at 0x7f0bb4860fd0>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method BatchNormalization.call of <tensorflow.python.layers.normalization.BatchNormalization object at 0x7f0bb4860fd0>>: AssertionError: Bad argument number for Name: 3, expecting 4
W0210 18:25:07.184139 139689548273472 ag_logging.py:145] Entity <bound method Conv.call of <tensorflow.python.layers.convolutional.Conv2D object at 0x7f0bb4e452e8>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: conv

...


W0210 18:25:34.256114 140201821710144 deprecation_wrapper.py:119] From /home/jayson/projects/ssd-dag/code/models/research/object_detection/predictors/convolutional_box_predictor.py:150: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.

INFO:tensorflow:depth of additional conv before box predictor: 0
I0210 18:25:34.256216 140201821710144 convolutional_box_predictor.py:151] depth of additional conv before box predictor: 0
W0210 18:25:34.336426 140201821710144 ag_logging.py:145] Entity <bound method Conv.call of <tensorflow.python.layers.convolutional.Conv2D object at 0x7f82f9e29b00>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Conv.call of <tensorflow.python.layers.convolutional.Conv2D object at 0x7f82f9e29b00>>: AssertionError: Bad argument number for 

W0210 18:25:34.649264 140201821710144 ag_logging.py:145] Entity <bound method Conv.call of <tensorflow.python.layers.convolutional.Conv2D object at 0x7f82f9d07be0>> could not be transformed and will be executed as-is. Please report this to the AutgoGraph team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output. Cause: converting <bound method Conv.call of <tensorflow.python.layers.convolutional.Conv2D object at 0x7f82f9d07be0>>: AssertionError: Bad argument number for Name: 3, expecting 4
INFO:tensorflow:depth of additional conv before box predictor: 0
I0210 18:25:34.651467 140201821710144 convolutional_box_predictor.py:151] depth of additional conv before box predictor: 0
W0210 18:25:34.693728 140201821710144 ag_logging.py:145] Entity <bound method Conv.call of <tensorflow.python.layers.convolutional.Conv2D object at 0x7f82f9d303c8>> could not be transformed and will be executed as-is. Please report this to the AutgoGrap

2020-02-10 18:25:34.947464: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1005] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-02-10 18:25:34.947857: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x563fd3094420 executing computations on platform CUDA. Devices:
2020-02-10 18:25:34.947873: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): GeForce RTX 2080 Ti, Compute Capability 7.5
2020-02-10 18:25:34.947964: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1005] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-02-10 18:25:34.948235: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1640] Found device 0 with properties: 
name: GeForce RTX 2080 Ti major: 7 minor: 5 memoryClockRate(GHz): 1.635
pciBusID: 0000:01:00.0
2020-02-10 18:25:34.948257: I tensorflo

Instructions for updating:
Use standard file APIs to check for files with this prefix.
W0210 18:25:36.745912 140201821710144 deprecation.py:323] From /home/jayson/anaconda3/envs/tf114/lib/python3.6/site-packages/tensorflow/python/tools/freeze_graph.py:127: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
2020-02-10 18:25:37.110256: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1005] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-02-10 18:25:37.110440: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1640] Found device 0 with properties: 
name: GeForce RTX 2080 Ti major: 7 minor: 5 memoryClockRate(GHz): 1.635
pciBusID: 0000:01:00.0
2020-02-10 18:25:37.110467: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] S

2020-02-10 18:25:39.342896: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcuda.so.1
2020-02-10 18:25:39.360957: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1005] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-02-10 18:25:39.361239: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1640] Found device 0 with properties: 
name: GeForce RTX 2080 Ti major: 7 minor: 5 memoryClockRate(GHz): 1.635
pciBusID: 0000:01:00.0
2020-02-10 18:25:39.361332: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcudart.so.10.0
2020-02-10 18:25:39.361895: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcublas.so.10.0
2020-02-10 18:25:39.362413: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcu

## Saved (output) Graphs

These models will now behave differently.   The API is different and the performance is different.   Furthermore the dependencies are different.  The JSON that is returned will not be compatible.   So, make sure you know what your doing.

### TensorFlow - Frozen Graph
Look for examples of using a Tensor frozen graph.  Look at notebook:  DetectModel_Step1_Local_TensorFlowFrozenGraph

### TensorFlow - TFLite
Look for examples of using TensorFlow Lite.   Look at notebook: DetectMOdel_Step1_Local_TFLite



In [6]:
! ls {TENSORFLOW_DIR} -l

total 58612
-rw-r--r-- 1 jayson jayson       77 Feb 10 18:25 checkpoint
-rw-r--r-- 1 jayson jayson 29362103 Feb 10 18:25 frozen_inference_graph.pb
-rw-r--r-- 1 jayson jayson 27381492 Feb 10 18:25 model.ckpt.data-00000-of-00001
-rw-r--r-- 1 jayson jayson    14948 Feb 10 18:25 model.ckpt.index
-rw-r--r-- 1 jayson jayson  3239277 Feb 10 18:25 model.ckpt.meta
-rw-r--r-- 1 jayson jayson     4375 Feb 10 18:25 pipeline.config
drwxr-xr-x 3 jayson jayson     4096 Feb 10 18:25 saved_model


In [7]:
! ls {TFLITE_DIR} -l

total 110568
-rw-r--r-- 1 jayson jayson  6899200 Feb 10 18:25 output_tflite_graph.tflite
-rw-r--r-- 1 jayson jayson 27693913 Feb 10 18:25 tflite_graph.pb
-rw-r--r-- 1 jayson jayson 78622342 Feb 10 18:25 tflite_graph.pbtxt
